# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
        print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Table for Query 1

The table **song_length** use **sessionId** and **itemInSession** as composited PARTITION KEY because the query use **sessionId** and **itemInSession** as filter.

In [14]:
drop_query = "DROP TABLE IF EXISTS song_length"
create_query="""
CREATE TABLE IF NOT EXISTS song_length
(sessionId int, itemInSession int, song_title text, artist text, length double,
PRIMARY KEY( (sessionId,itemInSession)) )
"""

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length (sessionId, itemInSession, song_title, artist, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
q = """
SELECT artist, song_title, length
FROM song_length
WHERE sessionId = 338 AND itemInSession = 4
"""

try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Table for Query 2

The table **song_playlist_session** use **sessionId** as PARTITION KEY.
**userid** and **sessionid** as CLUSTERING KEYs because the output of the query needs to sorted by **itemInSession** and filtered by **userid** and **sessionid**.

In [17]:
## Step 1 Create Table
drop_query = "DROP TABLE IF EXISTS song_playlist_session"
create_query="""
CREATE TABLE IF NOT EXISTS song_playlist_session
(sessionId int, userId int, itemInSession int, firstName text, lastName text, artist text, song_title text,
PRIMARY KEY(sessionId, userId , itemInSession) )
"""

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)

In [18]:
## Step 2 Insert Data

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_playlist_session (sessionId, userId, itemInSession, firstName, lastName, artist, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]),int(line[10]),int(line[3]), line[1], line[4], line[0], line[9]) )

In [19]:
## Step 3 test
q = """
SELECT artist, song_title, firstName, lastName, itemInSession
FROM song_playlist_session
WHERE sessionId = 182 AND userId = 10
"""

try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)
Row(artist='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)


### Table for Query 3


The query wants to know who listens to a specific song.
So table **song_listener** takes **song_title**, **firstName** and **lastName** as PRIMARY KEY, where **song_title** as PARTITION KEY and the others are CLUSTERING KEYs.

In [17]:
## Step 1 Create Table
drop_query = "DROP TABLE IF EXISTS song_listener"
create_query="""
CREATE TABLE IF NOT EXISTS song_listener
(song_title text, firstName text, lastName text,
PRIMARY KEY(song_title,firstName, lastName) )
"""

try:
    session.execute(drop_query)
    session.execute(create_query)
except Exception as e:
    print(e)
                    

In [18]:
## Step 2 Insert Data

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_listener (song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s)"

        session.execute(query, (line[9], line[1], line[4]) )

In [19]:
## Step 3 test

q = """
SELECT firstName, lastName, song_title
FROM song_listener
WHERE song_title = 'All Hands Against His Own'
"""

try:
    rows = session.execute(q)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch', song_title='All Hands Against His Own')
Row(firstname='Sara', lastname='Johnson', song_title='All Hands Against His Own')
Row(firstname='Tegan', lastname='Levine', song_title='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [75]:
drop1 = "DROP TABLE IF EXISTS song_length"
drop2 = "DROP TABLE IF EXISTS song_playlist_session"
drop3 = "DROP TABLE IF EXISTS song_listener"


try:
    session.execute(drop1)
    session.execute(drop2)
    session.execute(drop3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [76]:
session.shutdown()
cluster.shutdown()